##### Copyright 2022 The TensorFlow Authors.

In [13]:
import pandas as pd
df = pd.read_csv('data/TFRS-ranking/ratings.csv')
df

,username,ID,rating,user_id
0,akshaygahlot73,131689,2,284477248865
1,akshaygahlot73,129692,6,284477248865
2,akshaygahlot73,129689,7,284477248865
3,akshaygahlot73,129052,4,284477248865
4,akshaygahlot73,129130,7,284477248865
...,...,...,...,...
4883939,shubhamgupta.62000,108892,10,2311063523943
4883940,shubhamgupta.62000,112720,9,2311063523943
4883941,shubhamgupta.62000,109920,2,2311063523943
4883942,shubhamgupta.62000,18163,10,2311063523943


In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [11]:
df = df.drop('Unnamed: 0', axis = 1)

# Using TensorFlow Recommenders with TFX

***A tutorial to train a TensorFlow Recommenders ranking model as a [TFX pipeline](https://www.tensorflow.org/tfx).***

In [12]:
df.to_csv('data/TFRS-ranking/ratings.csv', index = False)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/recommenders/examples/ranking_tfx"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/ranking_tfx.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders/blob/main/docs/examples/ranking_tfx.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/recommenders/docs/examples/ranking_tfx.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In this notebook-based tutorial, we will create and run a [TFX pipeline](https://www.tensorflow.org/tfx)
to train a ranking model to predict movie ratings using TensorFlow Recommenders (TFRS).
The pipeline will consist of three essential TFX components: ExampleGen,
Trainer and Pusher. The pipeline includes the most minimal ML workflow like
importing data, training a model and exporting the trained TFRS ranking model.

## Set Up
We first need to install the TFX Python package and download
the dataset which we will use for our model.

### Upgrade Pip

To avoid upgrading Pip in a system when running locally,
check to make sure that we are running in Colab.
Local systems can of course be upgraded separately.

In [3]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

### Install TFX


In [2]:
!pip install -U tfx
!pip install -U tensorflow-recommenders

  Using cached tfx-1.9.0-py3-none-any.whl (2.5 MB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached tensorflow_model_analysis-0.40.0-py3-none-any.whl (1.8 MB)
  Using cached ml_pipelines_sdk-1.9.0-py3-none-any.whl (1.3 MB)
  Using cached google_api_python_client-1.12.11-py2.py3-none-any.whl (62 kB)
  Using cached tensorflow-2.9.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (511.7 MB)
  Using cached google_apitools-0.5.32-py3-none-any.whl (135 kB)
  Using cached tensorflow_data_validation-1.9.0-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.5 MB)
  Using cached kubernetes-12.0.1-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorflow_transform-1.9.0-py3-none-any.whl (436 kB)
  Using cached tensorflow_serving_api-2.9.1-py2.py3-none-any.whl (37 kB)
  Using cached docker-4.4.4-py2.py3-none-any.whl (147 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 210.4 kB/s eta 0:00:00 kB/s eta 0:00:01:01
  Using cached attrs-20.3.0-py2.py3-n

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 1.1 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.4/234.4 kB 710.6 kB/s eta 0:00:001m759.1 kB/s eta 0:00:01
  Using cached grpcio_gcp-0.2.2-py2.py3-none-any.whl (9.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.2/180.2 kB 1.4 MB/s eta 0:00:002.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 531.7 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.2/118.2 kB 1.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 1.3 MB/s eta 0:00:001.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 905.3 kB/s eta 0:00:0031m1.8 MB/s eta 0:00:01
  Using cached protobuf-3.20.1-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 329.7 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 264.3 kB/s eta 0:00:001m272.8 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 898.9 kB/s eta 0:00:001m998.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 743.5 kB/s eta 0:00:00MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 916.0 kB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.8/793.8 kB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 1.7 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 294.4 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 329.1 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 340.0 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 341.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 321.0 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 304.3 kB/s eta 0:00:001m308.4 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.7/211.7 kB 291.0 kB/s eta 0:00:001m297.7 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 235.9 kB/s eta 0:00:00m eta 0:00:010:00:02
  Using cached Send2Trash-1.8.0-py3-none-any.whl (18 kB)
  Using cached argon2_cffi-21.3.0-py3-none-any.whl (14 kB)
  Using cached prometheus_client-0.14.1-py3-none-any.whl (59 kB)
  Using cached nbconvert-6.5.0-py3-none-any.whl (561 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 208.1 kB/s eta 0:00:000:00:0136m-:--:--
  Using cached beautifulsoup4-4.11.1-py3-none-any.whl (128 kB)
  Using cached tinycss2-1.1.1-py3-none-any.whl (21 kB)
  Using cached jupyterlab_pygments-0.2.2-py2.py3-none-any.whl (21 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 187.5 kB

  Created wheel for google-apitools: filename=google_apitools-0.5.31-py3-none-any.whl size=131041 sha256=5348e70752c215a00168565ab0d4ac9d0f6440beec9ab9fd36d2fac89a671cdb
  Stored in directory: /home/padma/.cache/pip/wheels/6c/f8/60/b9e91899dbaf25b6314047d3daee379bdd8d61b1dc3fd5ec7f
  Created wheel for pyfarmhash: filename=pyfarmhash-0.3.2-cp39-cp39-linux_x86_64.whl size=14328 sha256=5d7d8e136d2f548fd952bb5507996d526ac21deb7d3462cfc84c61c5caed18fc
  Stored in directory: /home/padma/.cache/pip/wheels/de/2b/b1/c541160670d70f4b08c4786f4e155337d4baeaa3e01d9d1400
Successfully built google-apitools pyfarmhash
  Attempting uninstall: joblib
    Found existing installation: joblib 1.0.1
    Uninstalling joblib-1.0.1:
      Successfully uninstalled joblib-1.0.1
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
  Attempting uninstall: traitlets
  

### Did you restart the runtime?

If you are using Google Colab, the first time that you run
the cell above, you must restart the runtime by clicking
above "RESTART RUNTIME" button or using "Runtime > Restart
runtime ..." menu. This is because of the way that Colab
loads packages.

Before we define the pipeline, we need to write the model code for the
Trainer component and save it in a file.

In [3]:
!pip install -U apache-beam[interactive]

  Using cached jupyter_client-6.1.12-py3-none-any.whl (112 kB)
  Using cached google_cloud_dataproc-3.1.1-py2.py3-none-any.whl (186 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 155.1 kB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached timeloop-1.0.2.tar.gz (2.9 kB)
  Preparing metadata (setup.py) ... done


  Created wheel for timeloop: filename=timeloop-1.0.2-py3-none-any.whl size=3720 sha256=9935070b074aa55b65dff76b3191ac0cff40e2e8ba233b4b7defd367fca9f51a
  Stored in directory: /home/padma/.cache/pip/wheels/63/47/01/8e48745e2b92e8a78dc988d4e9404e4f10ccdcd207dc0cad69
Successfully built timeloop
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 7.2.2
    Uninstalling jupyter-client-7.2.2:
      Successfully uninstalled jupyter-client-7.2.2
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-model-analysis 0.40.0 requires ipython<8,>=7, but you have ipython 8.4.0 which is incompatible.


Check the TensorFlow and TFX versions.

In [ ]:
!pip install tensorflow-gpu

In [19]:
!pip -m install --upgrade pip


Usage:   
  pip <command> [options]

no such option: -m


In [1]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.9.1
TFX version: 1.9.0


### Set up variables

There are some variables used to define a pipeline. You can customize these
variables as you want. By default all output from the pipeline will be
generated under the current directory. Instead of using the SchemaGen component to generate a schema, for this
tutorial we will create a hardcoded schema.

In [2]:
import os

PIPELINE_NAME = 'TFRS-ranking'

# Directory where MovieLens 100K rating data lives
DATA_ROOT = os.path.join('data', PIPELINE_NAME)
# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### Prepare example data
Since TFX does not currently support TensorFlow Datasets API, we will download the MovieLens 100K dataset manually for use in our TFX pipeline. The dataset we
are using is
[MovieLens 100K Dataset](https://grouplens.org/datasets/movielens/100k/).

There are four numeric features in this dataset:

- userId
- movieId
- rating
- timestamp

We will build a ranking model which predicts the `rating` of the movies. We will not use the `timestamp` feature.

Because TFX ExampleGen reads inputs from a directory, we need to create a
directory and copy dataset to it.

In [25]:
# !wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
# !mkdir -p {DATA_ROOT}
# !unzip ml-100k.zip
# !echo 'username,ID,rating' > {DATA_ROOT}/ratings.csv
# !sed 's/\t/,/g' ml-100k/u.data >> {DATA_ROOT}/ratings.csv

Take a quick look at the CSV file.

In [14]:
!head {DATA_ROOT}/ratings.csv

username,ID,rating,user_id
akshaygahlot73,131689,2,284477248865
akshaygahlot73,129692,6,284477248865
akshaygahlot73,129689,7,284477248865
akshaygahlot73,129052,4,284477248865
akshaygahlot73,129130,7,284477248865
akshaygahlot73,125932,1,284477248865
akshaygahlot73,125529,4,284477248865
akshaygahlot73,125526,10,284477248865
akshaygahlot73,125523,7,284477248865


You should be able to see four values. For example, the first example means user '196' gives a rating of 3 to movie '242'.

## Create a pipeline

TFX pipelines are defined using Python APIs. We will define a pipeline which
consists of following three components.
- CsvExampleGen: Reads in data files and convert them to TFX internal format
for further processing. There are multiple
[ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen)s for various
formats. In this tutorial, we will use CsvExampleGen which takes CSV file input.
- Trainer: Trains an ML model.
[Trainer component](https://www.tensorflow.org/tfx/guide/trainer) requires a
model definition code from users. You can use TensorFlow APIs to specify how to
train a model and save it in a _saved_model_ format.
- Pusher: Copies the trained model outside of the TFX pipeline.
[Pusher component](https://www.tensorflow.org/tfx/guide/pusher) can be thought
of an deployment process of the trained ML model.

Before actually define the pipeline, we need to write a model code for the
Trainer component first.

### Write model training code

We will build a simple ranking model to predict movie ratings. This model training code will be saved to a separate file.

In this tutorial we will use
[Generic Trainer](https://www.tensorflow.org/tfx/guide/trainer#generic_trainer)
of TFX which support Keras-based models. You need to write a Python file
containing `run_fn` function, which is the entrypoint for the `Trainer`
component.

In [15]:
_trainer_module_file = 'tfrs_ranking_trainer.py'

The ranking model we use is almost exactly the same as in the [Basic Ranking](https://www.tensorflow.org/recommenders/examples/basic_ranking) tutorial. The only difference is that we use movie IDs instead of movie titles in the candidate tower.

In [16]:
%%writefile {_trainer_module_file}

from typing import Dict, Text
from typing import List
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow_recommenders as tfrs
from tensorflow_transform.tf_metadata import schema_utils
from tfx import v1 as tfx
from tfx_bsl.public import tfxio

_FEATURE_KEYS = ['user_id', 'ID']
_LABEL_KEY = 'rating'

_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
        for feature in _FEATURE_KEYS
    }, _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}
df = pd.read_csv('data/TFRS-ranking/ratings.csv')
user_count = len(df['user_id'].unique())
problem_count = len(df['ID'].unique())

class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    unique_user_ids = np.array(range(user_count)).astype(str)
    unique_problem_ids = np.array(range(problem_count)).astype(str)

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,), name='user_id', dtype=tf.int64),
        tf.keras.layers.Lambda(lambda x: tf.as_string(x)),
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(
            len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for problems.
    self.problem_embeddings = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,), name='ID', dtype=tf.int64),
        tf.keras.layers.Lambda(lambda x: tf.as_string(x)),
        tf.keras.layers.StringLookup(
            vocabulary=unique_problem_ids, mask_token=None),
        tf.keras.layers.Embedding(
            len(unique_problem_ids) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

  def call(self, inputs):

    user_id, problem_id = inputs

    user_embedding = self.user_embeddings(user_id)
    problem_embedding = self.problem_embeddings(problem_id)

    return self.ratings(tf.concat([user_embedding, problem_embedding], axis=2))


class problemlensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()])

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model((features['user_id'], features['ID']))

  def compute_loss(self,
                   features: Dict[Text, tf.Tensor],
                   training=False) -> tf.Tensor:

    labels = features[1]
    rating_predictions = self(features[0])

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 256) -> tf.data.Dataset:
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  return problemlensModel()


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files, fn_args.data_accessor, schema, batch_size=8192)
  eval_dataset = _input_fn(
      fn_args.eval_files, fn_args.data_accessor, schema, batch_size=4096)

  model = _build_keras_model()

  model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      epochs = 3,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  model.save(fn_args.serving_model_dir)

Writing tfrs_ranking_trainer.py


Now you have completed all preparation steps to build the TFX pipeline.

### Write a pipeline definition

We define a function to create a TFX pipeline. A `Pipeline` object
represents a TFX pipeline which can be run using one of pipeline
orchestration systems that TFX supports.


In [17]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=12),
      eval_args=tfx.proto.EvalArgs(num_steps=24))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

## Run the pipeline

TFX supports multiple orchestrators to run pipelines.
In this tutorial we will use `LocalDagRunner` which is included in the TFX
Python package and runs pipelines on local environment.

Now we create a `LocalDagRunner` and pass a `Pipeline` object created from the
function we already defined.

The pipeline runs directly and you can see logs for the progress of the pipeline including ML model training.

In [18]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

INFO:absl:Generating ephemeral wheel package for '/home/padma/Desktop/Coderecs/tfrs_ranking_trainer.py' (including modules: ['tfrs_ranking_trainer']).
INFO:absl:User module package has hash fingerprint version a316e354e9f0b2767a29ad95476ac0421c04d4b94e8a1208c15b8e9ad1b35932.
INFO:absl:Executing: ['/home/padma/anaconda3/envs/tf_gpu/bin/python', '/tmp/tmp5ez91m6z/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmphigo297y', '--dist-dir', '/tmp/tmpxvgmqbg2']
INFO:absl:Successfully built user code wheel distribution at 'pipelines/TFRS-ranking/_wheels/tfx_user_code_Trainer-0.0+a316e354e9f0b2767a29ad95476ac0421c04d4b94e8a1208c15b8e9ad1b35932-py3-none-any.whl'; target user module is 'tfrs_ranking_trainer'.
INFO:absl:Full user module path is 'tfrs_ranking_trainer@pipelines/TFRS-ranking/_wheels/tfx_user_code_Trainer-0.0+a316e354e9f0b2767a29ad95476ac0421c04d4b94e8a1208c15b8e9ad1b35932-py3-none-any.whl'
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  va

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying tfrs_ranking_trainer.py -> build/lib
installing to /tmp/tmphigo297y
running install
running install_lib
copying build/lib/tfrs_ranking_trainer.py -> /tmp/tmphigo297y
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmphigo297y/tfx_user_code_Trainer-0.0+a316e354e9f0b2767a29ad95476ac0421c04d4b94e8a1208c15b8e9ad1b35932-py3.9.egg-info
running install_scripts
creating /tmp/tmphigo297y/tfx_user_code_Trainer-0.0+a316e354e9f0b2767

INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Going to run a new execution 1
INFO:absl:Going to run a new execution: ExecutionInfo(execution_id=1, input_dict={}, output_dict=defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "pipelines/TFRS-ranking/CsvExampleGen/examples/1"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:153754296,xor_checksum:1658150224,sum_checksum:1658150224"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "TFRS-ranking:2022-07-18T18:47:40.001594:CsvExampleGen:examples:0"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
name: "TFRS-ranking:2022-07-18T18:47:40.001594:CsvExampleGen:examples:0"
, artifact_type: name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {

INFO:absl:Processing input csv data data/TFRS-ranking/* to TFExample.
E0718 18:47:41.766136707   14645 fork_posix.cc:76]           Other threads are currently calling into gRPC, skipping fork() handlers
INFO:absl:Examples generated.
INFO:absl:Value type <class 'NoneType'> of key version in exec_properties is not supported, going to drop it
INFO:absl:Value type <class 'list'> of key _beam_pipeline_args in exec_properties is not supported, going to drop it
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 1 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "pipelines/TFRS-ranking/CsvExampleGen/examples/1"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:153754296,xor_checksum:1658150224,sum_checksum:1658150224"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "TFRS-ranking

INFO:absl:Train on the 'train' split when train_args.splits is not set.
INFO:absl:Evaluate on the 'eval' split when eval_args.splits is not set.
INFO:absl:udf_utils.get_fn {'module_path': 'tfrs_ranking_trainer@pipelines/TFRS-ranking/_wheels/tfx_user_code_Trainer-0.0+a316e354e9f0b2767a29ad95476ac0421c04d4b94e8a1208c15b8e9ad1b35932-py3-none-any.whl', 'train_args': '{\n  "num_steps": 12\n}', 'eval_args': '{\n  "num_steps": 24\n}', 'custom_config': 'null'} 'run_fn'
INFO:absl:Installing 'pipelines/TFRS-ranking/_wheels/tfx_user_code_Trainer-0.0+a316e354e9f0b2767a29ad95476ac0421c04d4b94e8a1208c15b8e9ad1b35932-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/home/padma/anaconda3/envs/tf_gpu/bin/python', '-m', 'pip', 'install', '--target', '/tmp/tmph9wnb5bd', 'pipelines/TFRS-ranking/_wheels/tfx_user_code_Trainer-0.0+a316e354e9f0b2767a29ad95476ac0421c04d4b94e8a1208c15b8e9ad1b35932-py3-none-any.whl']
E0718 18:56:36.618800029   14645 fork_posix.cc:76]           Other threads are

...

INFO:absl:Successfully installed 'pipelines/TFRS-ranking/_wheels/tfx_user_code_Trainer-0.0+a316e354e9f0b2767a29ad95476ac0421c04d4b94e8a1208c15b8e9ad1b35932-py3-none-any.whl'.
INFO:absl:Training model.
INFO:absl:Feature ID has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature rating has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
2022-07-18 18:56:39.647061: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-18 18:56:40.215346: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7071 MB memory:  -> device: 0, name: Quadro P4000, pci bus 

Epoch 1/3
12/12 [==============================] - 6s 130ms/step - root_mean_squared_error: 3.9801 - loss: 15.4937 - regularization_loss: 0.0000e+00 - total_loss: 15.4937 - val_root_mean_squared_error: 3.1391 - val_loss: 9.1427 - val_regularization_loss: 0.0000e+00 - val_total_loss: 9.1427
Epoch 2/3
12/12 [==============================] - 1s 87ms/step - root_mean_squared_error: 3.1544 - loss: 9.8621 - regularization_loss: 0.0000e+00 - total_loss: 9.8621 - val_root_mean_squared_error: 3.1506 - val_loss: 8.8569 - val_regularization_loss: 0.0000e+00 - val_total_loss: 8.8569
Epoch 3/3
12/12 [==============================] - 2s 137ms/step - root_mean_squared_error: 3.0255 - loss: 9.2010 - regularization_loss: 0.0000e+00 - total_loss: 9.2010 - val_root_mean_squared_error: 3.1192 - val_loss: 8.9075 - val_regularization_loss: 0.0000e+00 - val_total_loss: 8.9075


INFO:tensorflow:Assets written to: pipelines/TFRS-ranking/Trainer/model/2/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/TFRS-ranking/Trainer/model/2/Format-Serving/assets
INFO:absl:Training complete. Model written to pipelines/TFRS-ranking/Trainer/model/2/Format-Serving. ModelRun written to pipelines/TFRS-ranking/Trainer/model_run/2
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 2 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model': [Artifact(artifact: uri: "pipelines/TFRS-ranking/Trainer/model/2"
custom_properties {
  key: "name"
  value {
    string_value: "TFRS-ranking:2022-07-18T18:47:40.001594:Trainer:model:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.9.0"
  }
}
name: "TFRS-ranking:2022-07-18T18:47:40.001594:Trainer:model:0"
, artifact_type: name: "Model"
base_type: MODEL
)], 'model_run': [Artifact(artifact: uri: "pipelines/TFRS-ranking/Trainer/model_run/2"
custom_properties {
  key: "name"
  value {
    string_value: 

INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component Pusher is finished.


You should see "INFO:absl:Component Pusher is finished." at the end of the
logs if the pipeline finished successfully. Because `Pusher` component is the
last component of the pipeline.

The pusher component pushes the trained model to the `SERVING_MODEL_DIR` which
is the `serving_model/TFRS-ranking` directory if you did not change the
variables in the previous steps. You can see the result from the file browser
in the left-side panel in Colab, or using the following command:

In [12]:
# List files in created model directory.
!ls -R {SERVING_MODEL_DIR}

1654925577 1655570863

serving_model/TFRS-ranking/1654925577:
assets            keras_metadata.pb saved_model.pbtxt variables

serving_model/TFRS-ranking/1654925577/assets:

serving_model/TFRS-ranking/1654925577/variables:
variables.data-00000-of-00001 variables.index

serving_model/TFRS-ranking/1655570863:
assets            keras_metadata.pb saved_model.pb    variables

serving_model/TFRS-ranking/1655570863/assets:

serving_model/TFRS-ranking/1655570863/variables:
variables.data-00000-of-00001 variables.index


In [12]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.9.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (511.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 1.3 MB/s eta 0:00:00m eta 0:00:010:01:010m
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached absl_py-1.1.0-py3-none-any.whl (123 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 489.7 kB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.0 kB/s eta 0:00:00m eta 0:00:010:00:01m
  Using cached libclang-14.0.1-py2.py3-none-manylinux1_x86_64.whl (14.5 MB)
  Using cached flatbuffers-1.12-py2.py3-none-a

Now we can test the ranking model by computing predictions for a user and a movie:

In [20]:
import glob
# Load the latest model for testing
loaded = tf.saved_model.load(max(glob.glob(os.path.join(SERVING_MODEL_DIR, '*/')), key=os.path.getmtime))
print(loaded({'user_id': [[1291971074983]], 'ID': [[132332]]}).numpy())

[[[3.7295794]]]


This concludes the TensorFlow Recommenders + TFX tutorial.